In [1]:
#!pip install python-dotenv decorator==4.3 networkx
#!pip install pycoingecko
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import pprint


%matplotlib inline

In [2]:
load_dotenv('../.env')

True

In [3]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [4]:
coin_market = cg.get_coins_markets(vs_currency='usd')
coin_market_df= pd.DataFrame(coin_market, columns=['id', 'current_price', 'high_24h' , 'low_24h'])
coin_market_df.set_index('id', inplace=True)
coin_market_df

,current_price,high_24h,low_24h
id,,,
bitcoin,54895.000000,56056.000000,53909.000000
ethereum,3604.220000,3674.850000,3556.800000
cardano,2.260000,2.320000,2.230000
tether,1.000000,1.010000,0.994718
binancecoin,423.990000,442.820000,417.420000
...,...,...,...
mina-protocol,4.600000,4.720000,4.480000
bitcoin-gold,66.580000,71.220000,65.560000
magic-internet-money,1.010000,1.030000,0.991758


In [5]:
btc_history = cg.get_coin_market_chart_by_id(id= 'bitcoin', vs_currency = 'usd', days = '1000')
btc_history_df = pd.DataFrame(btc_history)
btc_history_df

,prices,market_caps,total_volumes
0,"[1547424000000, 3495.832311307438]","[1547424000000, 61050348901.5798]","[1547424000000, 2675674829.818869]"
1,"[1547510400000, 3648.04828565649]","[1547510400000, 63758350560.63251]","[1547510400000, 3101461657.547051]"
2,"[1547596800000, 3567.801243199206]","[1547596800000, 62348810149.49811]","[1547596800000, 2953789408.2270346]"
3,"[1547683200000, 3597.6473233295915]","[1547683200000, 62969304894.27945]","[1547683200000, 2753646523.928816]"
4,"[1547769600000, 3620.9812811457246]","[1547769600000, 63320099663.395294]","[1547769600000, 3049367561.0957274]"
...,...,...,...
996,"[1633478400000, 51668.99422435564]","[1633478400000, 973040988102.9114]","[1633478400000, 35480330025.02791]"
997,"[1633564800000, 55415.00574202895]","[1633564800000, 1044675252844.3663]","[1633564800000, 48462388015.80152]"
998,"[1633651200000, 53894.59599452107]","[1633651200000, 1015246512238.866]","[1633651200000, 36549815857.28183]"
999,"[1633737600000, 54010.93747374405]","[1633737600000, 1016650501581.1074]","[1633737600000, 33935485256.826164]"


In [6]:
def extract_volume(x):
    return x[0]
def extract_price(x):
    return x[1]

btc_history_df["volume"] = btc_history_df['prices'].apply(extract_volume)
btc_history_df["price"] = btc_history_df['prices'].apply(extract_price)
btc_history_df["market_cap"] = btc_history_df['market_caps'].apply(extract_price)
btc_history_df["total_volume"] = btc_history_df['total_volumes'].apply(extract_price)

In [7]:
btc_history_df = btc_history_df[['volume', 'price',
       'market_cap', 'total_volume']]

In [8]:
btc_history_df

,volume,price,market_cap,total_volume
0,1547424000000,3495.832311,6.105035e+10,2.675675e+09
1,1547510400000,3648.048286,6.375835e+10,3.101462e+09
2,1547596800000,3567.801243,6.234881e+10,2.953789e+09
3,1547683200000,3597.647323,6.296930e+10,2.753647e+09
4,1547769600000,3620.981281,6.332010e+10,3.049368e+09
...,...,...,...,...
996,1633478400000,51668.994224,9.730410e+11,3.548033e+10
997,1633564800000,55415.005742,1.044675e+12,4.846239e+10
998,1633651200000,53894.595995,1.015247e+12,3.654982e+10
999,1633737600000,54010.937474,1.016651e+12,3.393549e+10


In [9]:
eth_history = cg.get_coin_market_chart_by_id(id= 'ethereum', vs_currency = 'usd', days = '1000')
eth_history_df = pd.DataFrame(eth_history)
eth_history_df

,prices,market_caps,total_volumes
0,"[1547424000000, 114.76604584662175]","[1547424000000, 11970884149.01689]","[1547424000000, 1642681383.2372842]"
1,"[1547510400000, 127.40527329081972]","[1547510400000, 13312835355.826275]","[1547510400000, 2141032095.0550327]"
2,"[1547596800000, 119.81437324652917]","[1547596800000, 12511274618.87375]","[1547596800000, 2179445232.6582036]"
3,"[1547683200000, 122.24553850486993]","[1547683200000, 12790017531.356869]","[1547683200000, 2113608536.6644716]"
4,"[1547769600000, 121.92726666997898]","[1547769600000, 12732832650.46048]","[1547769600000, 1953391667.2246928]"
...,...,...,...
996,"[1633478400000, 3520.3422566393974]","[1633478400000, 415494260964.53455]","[1633478400000, 19024921944.49268]"
997,"[1633564800000, 3592.1761295000256]","[1633564800000, 423560000564.3804]","[1633564800000, 24709364134.235626]"
998,"[1633651200000, 3594.918357967647]","[1633651200000, 423610266229.8694]","[1633651200000, 21186801824.571068]"
999,"[1633737600000, 3558.549577083409]","[1633737600000, 419918377734.5956]","[1633737600000, 18144469093.779636]"


In [10]:
eth_history_df["volume"] = eth_history_df['prices'].apply(extract_volume)
eth_history_df["price"] = eth_history_df['prices'].apply(extract_price)
eth_history_df["market_cap"] = eth_history_df['market_caps'].apply(extract_price)
eth_history_df["total_volume"] = eth_history_df['total_volumes'].apply(extract_price)

In [11]:
eth_history_df = eth_history_df[['volume', 'price',
       'market_cap', 'total_volume']]
eth_history_df.head()

,volume,price,market_cap,total_volume
0,1547424000000,114.766046,1.197088e+10,1.642681e+09
1,1547510400000,127.405273,1.331284e+10,2.141032e+09
2,1547596800000,119.814373,1.251127e+10,2.179445e+09
3,1547683200000,122.245539,1.279002e+10,2.113609e+09
4,1547769600000,121.927267,1.273283e+10,1.953392e+09


In [12]:
lte_history = cg.get_coin_market_chart_by_id(id= 'litecoin', vs_currency = 'usd', days = '1000')
lte_history_df = pd.DataFrame(lte_history)
lte_history_df

,prices,market_caps,total_volumes
0,"[1547424000000, 29.6464825452393]","[1547424000000, 1778046097.7883253]","[1547424000000, 499281524.01884854]"
1,"[1547510400000, 31.744528225743903]","[1547510400000, 1908979231.600091]","[1547510400000, 562737057.7518421]"
2,"[1547596800000, 30.851094017836648]","[1547596800000, 1853454693.1427057]","[1547596800000, 514183004.657007]"
3,"[1547683200000, 31.39381775879312]","[1547683200000, 1885150175.08049]","[1547683200000, 493907069.895123]"
4,"[1547769600000, 31.432062198443145]","[1547769600000, 1887944820.258272]","[1547769600000, 570099751.8867725]"
...,...,...,...
996,"[1633478400000, 174.11810925008209]","[1633478400000, 11959486591.453056]","[1633478400000, 2624353783.5339026]"
997,"[1633564800000, 179.19993360488678]","[1633564800000, 12309753394.568789]","[1633564800000, 3225630004.9054756]"
998,"[1633651200000, 179.0563327253737]","[1633651200000, 12301131244.813486]","[1633651200000, 3515152317.772371]"
999,"[1633737600000, 175.96907129240822]","[1633737600000, 12090218245.97475]","[1633737600000, 3002931305.124391]"


In [13]:
lte_history_df["volume"] = lte_history_df['prices'].apply(extract_volume)
lte_history_df["price"] = lte_history_df['prices'].apply(extract_price)
lte_history_df["market_cap"] = lte_history_df['market_caps'].apply(extract_price)
lte_history_df["total_volume"] = lte_history_df['total_volumes'].apply(extract_price)

In [14]:
lte_history_df = lte_history_df[['volume', 'price',
       'market_cap', 'total_volume']]
lte_history_df.head()

,volume,price,market_cap,total_volume
0,1547424000000,29.646483,1.778046e+09,4.992815e+08
1,1547510400000,31.744528,1.908979e+09,5.627371e+08
2,1547596800000,30.851094,1.853455e+09,5.141830e+08
3,1547683200000,31.393818,1.885150e+09,4.939071e+08
4,1547769600000,31.432062,1.887945e+09,5.700998e+08


In [15]:
link_history = cg.get_coin_market_chart_by_id(id= 'chainlink', vs_currency = 'usd', days = '1000')
link_history_df = pd.DataFrame(link_history)
link_history_df

,prices,market_caps,total_volumes
0,"[1547424000000, 0.44288532560177163]","[1547424000000, 161391650.6145374]","[1547424000000, 11307556.12323578]"
1,"[1547510400000, 0.47754577316869523]","[1547510400000, 174017381.42313182]","[1547510400000, 20294178.44005211]"
2,"[1547596800000, 0.5052177589075403]","[1547596800000, 183962409.98381948]","[1547596800000, 29695908.089229587]"
3,"[1547683200000, 0.49123008251247313]","[1547683200000, 179092809.4370941]","[1547683200000, 19285693.19243903]"
4,"[1547769600000, 0.4766020502870565]","[1547769600000, 173678746.33172497]","[1547769600000, 11980676.526866283]"
...,...,...,...
996,"[1633478400000, 27.271440556604922]","[1633478400000, 12504224059.698664]","[1633478400000, 965956924.926952]"
997,"[1633564800000, 26.894354582472726]","[1633564800000, 12274829797.268402]","[1633564800000, 1278611228.7091522]"
998,"[1633651200000, 26.890237916117307]","[1633651200000, 12244178054.637278]","[1633651200000, 1174576942.443706]"
999,"[1633737600000, 26.266824635078514]","[1633737600000, 12042931529.36996]","[1633737600000, 895901735.8906263]"


In [16]:
link_history_df["volume"] = link_history_df['prices'].apply(extract_volume)
link_history_df["price"] = link_history_df['prices'].apply(extract_price)
link_history_df["market_cap"] = link_history_df['market_caps'].apply(extract_price)
link_history_df["total_volume"] = link_history_df['total_volumes'].apply(extract_price)

In [17]:
link_history_df = link_history_df[['volume', 'price',
       'market_cap', 'total_volume']]
link_history_df.head()

,volume,price,market_cap,total_volume
0,1547424000000,0.442885,1.613917e+08,1.130756e+07
1,1547510400000,0.477546,1.740174e+08,2.029418e+07
2,1547596800000,0.505218,1.839624e+08,2.969591e+07
3,1547683200000,0.491230,1.790928e+08,1.928569e+07
4,1547769600000,0.476602,1.736787e+08,1.198068e+07


In [18]:
ada_history = cg.get_coin_market_chart_by_id(id= 'cardano', vs_currency = 'usd', days = '1000')
ada_history_df = pd.DataFrame(ada_history)
ada_history_df

,prices,market_caps,total_volumes
0,"[1547424000000, 0.03957068318789578]","[1547424000000, 1228609037.9676828]","[1547424000000, 36923377.86663759]"
1,"[1547510400000, 0.0434219696959126]","[1547510400000, 1349921731.8845065]","[1547510400000, 46991904.800733194]"
2,"[1547596800000, 0.042207064638792174]","[1547596800000, 1312610496.9720254]","[1547596800000, 44720875.31453265]"
3,"[1547683200000, 0.04412438875413042]","[1547683200000, 1373439554.0270956]","[1547683200000, 55013701.89921163]"
4,"[1547769600000, 0.044726995298034124]","[1547769600000, 1391343341.6047573]","[1547769600000, 60179613.104692645]"
...,...,...,...
996,"[1633478400000, 2.237848940200704]","[1633478400000, 71779849396.20284]","[1633478400000, 1499391673.9169114]"
997,"[1633564800000, 2.2133070922060374]","[1633564800000, 70967214662.64716]","[1633564800000, 2144512119.753268]"
998,"[1633651200000, 2.279964070864212]","[1633651200000, 72987443161.14131]","[1633651200000, 2669936578.5982766]"
999,"[1633737600000, 2.2306779663351755]","[1633737600000, 71588795681.69351]","[1633737600000, 1521104723.9480116]"


In [19]:
ada_history_df["volume"] = ada_history_df['prices'].apply(extract_volume)
ada_history_df["price"] = ada_history_df['prices'].apply(extract_price)
ada_history_df["market_cap"] = ada_history_df['market_caps'].apply(extract_price)
ada_history_df["total_volume"] = ada_history_df['total_volumes'].apply(extract_price)

In [20]:
ada_history_df = ada_history_df[['volume', 'price',
       'market_cap', 'total_volume']]
ada_history_df.head()

,volume,price,market_cap,total_volume
0,1547424000000,0.039571,1.228609e+09,3.692338e+07
1,1547510400000,0.043422,1.349922e+09,4.699190e+07
2,1547596800000,0.042207,1.312610e+09,4.472088e+07
3,1547683200000,0.044124,1.373440e+09,5.501370e+07
4,1547769600000,0.044727,1.391343e+09,6.017961e+07


In [21]:
to_merge_dict = {
    'BTC': btc_history_df , 'ETH': eth_history_df,
    'LTE':lte_history_df,'LINK':link_history_df, 'ADA':ada_history_df }

In [22]:
#Coin Merged DataFrame
merged_coin_df = pd.concat(to_merge_dict.values(), keys=to_merge_dict.keys(), axis=1, join='inner')
merged_coin_df['date'] = ""
merged_coin_df.head()

BTC                                                     ETH  \
          volume        price    market_cap  total_volume         volume   
0  1547424000000  3495.832311  6.105035e+10  2.675675e+09  1547424000000   
1  1547510400000  3648.048286  6.375835e+10  3.101462e+09  1547510400000   
2  1547596800000  3567.801243  6.234881e+10  2.953789e+09  1547596800000   
3  1547683200000  3597.647323  6.296930e+10  2.753647e+09  1547683200000   
4  1547769600000  3620.981281  6.332010e+10  3.049368e+09  1547769600000   

                                                     LTE             ...  \
        price    market_cap  total_volume         volume      price  ...   
0  114.766046  1.197088e+10  1.642681e+09  1547424000000  29.646483  ...   
1  127.405273  1.331284e+10  2.141032e+09  1547510400000  31.744528  ...   
2  119.814373  1.251127e+10  2.179445e+09  1547596800000  30.851094  ...   
3  122.245539  1.279002e+10  2.113609e+09  1547683200000  31.393818  ...   
4  121.927267  1.273283e+10  1.953392e+09  1547769600000  31.432062  ...   

                          LINK                                        \
   total_volume         volume     price    market_cap  total_volume   
0  4.992815e+08  1547424000000  0.442885  1.613917e+08  1.130756e+07   
1  5.627371e+08  1547510400000  0.477546  1.740174e+08  2.029418e+07   
2  5.141830e+08  1547596800000  0.505218  1.839624e+08  2.969591e+07   
3  4.939071e+08  1547683200000  0.491230  1.790928e+08  1.928569e+07   
4  5.700998e+08  1547769600000  0.476602  1.736787e+08  1.198068e+07   

             ADA                                       date  
          volume     price    market_cap  total_volume       
0  1547424000000  0.039571  1.228609e+09  3.692338e+07       
1  1547510400000  0.043422  1.349922e+09  4.699190e+07       
2  1547596800000  0.042207  1.312610e+09  4.472088e+07       
3  1547683200000  0.044124  1.373440e+09  5.501370e+07       
4  1547769600000  0.044727  1.391343e+09  6.017961e+07       

[5 rows x 21 columns]

In [29]:
#Want to add dates to coresponding values
"""We are manually adding dates due to API Key not having dates. We need to have a common
column to concat for portfolios. dates would be the only thing to have a common column, but
we may run into an issue with crypto being calculated daily and not skipping non business days/hours
"""
rng = pd.date_range('2017-10-17','2021-10-08', freq='D') 
merged_coin_df['date'] = pd.to_datetime(merged_coin_df['date'])
merged_coin_df = merged_coin_df.set_index('date').reindex(rng, fill_value=0)

merged_coin_df.head()

KeyError: 'date'

In [ ]:
merged_coin_df.to_json('crypto_data.json')

In [ ]:
#Alpaca Keys
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [ ]:
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [ ]:
aapl_data = api.get_barset('AAPL',timeframe='1D', limit=1000,).df
aapl_data.tail()

In [ ]:
aa

In [ ]:
amzn_data = api.get_barset('AMZN',timeframe='1D', limit=1000,).df
amzn_data.head()

In [ ]:
ba_data = api.get_barset('BA',timeframe='1D', limit=1000,).df
ba_data.head()

In [ ]:
nvda_data = api.get_barset('NVDA',timeframe='1D', limit=1000,).df
nvda_data.head()

In [ ]:
brk_data = api.get_barset('BRK.A',timeframe='1D', limit=1000,).df
brk_data.head()

In [ ]:
gs_data = api.get_barset('GS',timeframe='1D', limit=1000,).df
gs_data.head()

In [ ]:
cop_data = api.get_barset('COP',timeframe='1D', limit=1000,).df
cop_data.head()

In [ ]:
gild_data = api.get_barset('GILD',timeframe='1D', limit=1000,).df
gild_data.head()

In [ ]:
ccoi_data = api.get_barset('CCOI',timeframe='1D', limit=1000,).df
ccoi_data.head()

In [ ]:
proc_data = api.get_barset('PG',timeframe='1D', limit=1000,).df
proc_data.head()

In [ ]:
# aapl_data.dropna(inplace=True)
# amzn_data.dropna(inplace=True)
# ba_data.dropna(inplace=True)
# nvda_data.dropna(inplace=True)
# brk_data.dropna(inplace=True)
# gs_data.dropna(inplace=True)
# cop_data.dropna(inplace=True) 
# gild_data.dropna(inplace=True)
# ccoi_data.dropna(inplace=True)
# proc_data.dropna(inplace=True)

In [ ]:
merged_stock_df= pd.concat([
    aapl_data, amzn_data, ba_data, nvda_data, brk_data,
    gs_data, cop_data, gild_data, ccoi_data, proc_data], axis='columns', join='inner')
#merged_stock_df = pd.concat(to_merge_dict.values())
#merged_stock_df.dropna(inplace=True)
merged_stock_df.head()

In [ ]:
merged_stock_df.to_json('stock_data.json')

In [ ]:
#Bond Data
#iSHares 20 year plus treasury
bond_data = api.get_barset(['TLT', 'VOOG', 'VGLT', 'VMBS', 'IWF', 'IGSB', 'GOVT'],timeframe='1D', limit=1000,).df


In [ ]:
bond_data.dropna(inplace=True)
bond_data.head()

In [ ]:
bond_data.to_json('bond_data.json')

In [ ]:
#Machine Learning for Investments
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Needed for decision tree visualization
import pydotplus
from IPython.display import Image

In [ ]:
#Calcualtions for Daily Return which help with creating a target for ML
merged_stock_df.loc[:, ('AAPL', 'Daily_Returns')] = merged_stock_df['AAPL']['close'].pct_change()
merged_stock_df.loc[:,('AMZN','Daily_Returns')] = merged_stock_df['AMZN']['close'].pct_change()
merged_stock_df.loc[:,('BA','Daily_Returns')] = merged_stock_df['BA']['close'].pct_change()
merged_stock_df.loc[:,('NVDA','Daily_Returns')] = merged_stock_df['NVDA']['close'].pct_change()
merged_stock_df.loc[:,('BRK.A','Daily_Returns')] = merged_stock_df['BRK.A']['close'].pct_change()
merged_stock_df.loc[:,('GS','Daily_Returns')] = merged_stock_df['GS']['close'].pct_change()
merged_stock_df.loc[:,('COP','Daily_Returns')] = merged_stock_df['COP']['close'].pct_change()
merged_stock_df.loc[:,('GILD','Daily_Returns')] = merged_stock_df['GILD']['close'].pct_change()
merged_stock_df.loc[:,('CCOI','Daily_Returns')] = merged_stock_df['CCOI']['close'].pct_change()
merged_stock_df.loc[:,('PG','Daily_Returns')] =merged_stock_df['PG']['close'].pct_change()
merged_stock_return_df=  merged_stock_df.copy()
merged_stock_return_df.dropna(inplace=True)

In [ ]:
merged_stock_return_df.loc[:, ('AAPL', 'Daily_Returns')] = merged_stock_df['AAPL']['close'].pct_change()
merged_stock_return_df.loc[:,('AMZN','Daily_Returns')] = merged_stock_df['AMZN']['close'].pct_change()
merged_stock_return_df.loc[:,('BA','Daily_Returns')] = merged_stock_df['BA']['close'].pct_change()
merged_stock_return_df.loc[:,('NVDA','Daily_Returns')] = merged_stock_df['NVDA']['close'].pct_change()
merged_stock_return_df.loc[:,('BRK.A','Daily_Returns')] = merged_stock_df['BRK.A']['close'].pct_change()
merged_stock_return_df.loc[:,('GS','Daily_Returns')] = merged_stock_df['GS']['close'].pct_change()
merged_stock_return_df.loc[:,('COP','Daily_Returns')] = merged_stock_df['COP']['close'].pct_change()
merged_stock_return_df.loc[:,('GILD','Daily_Returns')] = merged_stock_df['GILD']['close'].pct_change()
merged_stock_return_df.loc[:,('CCOI','Daily_Returns')] = merged_stock_df['CCOI']['close'].pct_change()
merged_stock_return_df.loc[:,('PG','Daily_Returns')] = merged_stock_df['PG']['close'].pct_change()

merged_stock_return_df.dropna(inplace=True)

In [ ]:
#iShares Russell 1000 Growth ETF
# iwf_data = api.get_barset('IWF',timeframe='1D', limit=1000,).df
# iwf_data.head()

In [ ]:
#iShares 1-5 year Investment grade bond
# igsb_data = api.get_barset('IGSB',timeframe='1D', limit=1000,).df
# igsb_data.head()

In [ ]:
#iShares US Treasury Bond ETF
# govt_data = api.get_barset('GOVT',timeframe='1D', limit=1000,).df
# govt_data.head()